# Final Project

## Link to dataset
Here is a <a href="https://www.kaggle.com/lepchenkov/usedcarscatalog" target="_blank">link</a> to the dataset on Kaggle.

## Possible Ideas
- Predict automatic or mechanical (manual) (*Perceptron, Logistic regression, SVM, k-NN*)
- Find similiarities in the vehicles (*k-Means*)
- Predict the duration of the listing given the make and model of the car (*Linear regression*)
- Reduce dimensionality of feature_1 through feature_9 columns to improve predictions (*PCA*)

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.linear_model import LinearRegression, LogisticRegression, Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('data/cars.csv')
df.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,True,False,True,True,False,False,False,False,True,7


In [3]:
df.columns

Index(['manufacturer_name', 'model_name', 'transmission', 'color',
       'odometer_value', 'year_produced', 'engine_fuel', 'engine_has_gas',
       'engine_type', 'engine_capacity', 'body_type', 'has_warranty', 'state',
       'drivetrain', 'price_usd', 'is_exchangeable', 'location_region',
       'number_of_photos', 'up_counter', 'feature_0', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'duration_listed'],
      dtype='object')

## Find Columns with Missing Values

In [4]:
cols = df.columns[df.isna().sum()>0]
print(f'Columns with missing values: {cols.values}')

Columns with missing values: ['engine_capacity']


Fill in the missing values with the mean of the column.

In [5]:
# df['engine_capacity'].fillna(np.mean(df['engine_capacity']), inplace=True)

In [6]:
df.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,True,False,True,True,False,False,False,False,True,7


## Drop certain rows and columns

In [7]:
electric_df = df[df['engine_type'] == 'electric']
len(electric_df)

10

Drop rows of electric vehicles because there are very few data samples of electric vehicles.

In [8]:
df.drop(electric_df.index, axis=0, inplace=True)

Drop ```"engine_fuel"``` column because it is too similar to ```"engine_type"```, which is what we are trying to predict.

In [9]:
df.drop('engine_fuel', axis=1, inplace=True)

## TODO 
Check columns that have very unequal data and decide to drop them.

In [10]:
df.engine_type.value_counts()

gasoline    25647
diesel      12874
Name: engine_type, dtype: int64

## Label Encode True/False values

Set boolean type columns to integers.

$\text{True}=1$

$\text{False}=0$

In [11]:
true_false = ['engine_has_gas', 'has_warranty', 'is_exchangeable']

df[true_false] = df[true_false].astype(int)

In [12]:
features = []
for i in range(10):
    features.append(f'feature_{i}')
    
df[features] = df[features].astype(int)

In [13]:
df.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_has_gas,engine_type,engine_capacity,body_type,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,0,gasoline,2.5,universal,...,1,1,1,0,1,0,1,1,1,16
1,Subaru,Outback,automatic,blue,290000,2002,0,gasoline,3.0,universal,...,1,0,0,1,1,0,0,0,1,83
2,Subaru,Forester,automatic,red,402000,2001,0,gasoline,2.5,suv,...,1,0,0,0,0,0,0,1,1,151
3,Subaru,Impreza,mechanical,blue,10000,1999,0,gasoline,3.0,sedan,...,0,0,0,0,0,0,0,0,0,86
4,Subaru,Legacy,automatic,black,280000,2001,0,gasoline,2.5,universal,...,1,0,1,1,0,0,0,0,1,7


## Function for Stratified k-Fold Cross Validation

In [14]:
def strat_kfold(X, y, clf):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    train_scores = []
    test_scores = []

    for train_idx, test_idx in skf.split(X, y):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]

        pipe = make_pipeline(transform_cols, clf)

        pipe.fit(X_train, y_train)

        train_scores.append(pipe.score(X_train, y_train))
        test_scores.append(pipe.score(X_test, y_test))

#     max_test_score_idx = np.argmax(test_scores)
    
#     max_train_score = round(train_scores[max_test_score_idx]*100, 2)
#     max_test_score = round(test_scores[max_test_score_idx]*100, 2)
    
#     max_train_score = round(max(train_scores)*100, 2)
    max_test_score = round(max(test_scores)*100, 2)
    
    average_train_score = round(np.mean(train_scores)*100, 2)
    average_test_score = round(np.mean(test_scores)*100, 2)

    print(f'Average training accuracy: {average_train_score}%',
      f'\nAverage testing Accuracy: {average_test_score}%')
    
    print(f'\nHighest testing accuracy: {max_test_score}%')

# Classification

Predict if the vehicle fuel is **"gasoline"** or **"diesel"**.

Label encode `engine_type` to use as label.

In [15]:
df['engine_type'] = df['engine_type'].apply(lambda x: 1 if x=='gasoline' else 0)

## `OneHotEncoder()` on categorical columns, `StandardScaler( )` on numerical data

In [16]:
# Columns with categorical data
cat_cols = ['manufacturer_name', 'model_name', 'transmission', 'color', 'year_produced', 
               'body_type', 'has_warranty', 'state', 'drivetrain', 'location_region']

# Columns with numerical data
num_cols = ['odometer_value', 'engine_capacity', 'price_usd', 'number_of_photos', 'up_counter', 'duration_listed']

transform_cols = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols),
    (StandardScaler(), num_cols),
    remainder='passthrough'
)

## Set X and y and split into test/train

Label: `engine_type`

In [17]:
X = df.drop('engine_type', axis=1)
y = df['engine_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train.shape, y_train.shape

((30816, 28), (30816,))

## Logistic Regression Pipeline

### Using `X_train` and `y_train`

In [18]:
logreg_pipe = make_pipeline(transform_cols, LogisticRegression(C=10, max_iter=4000))

logreg_pipe.fit(X_train, y_train)

training_score = round(logreg_pipe.score(X_train, y_train)*100, 2)
testing_score = round(logreg_pipe.score(X_test, y_test)*100, 2)

print(f'Training accuracy: {training_score}%',
      f'\nTesting Accuracy: {testing_score}%')

Training accuracy: 84.03% 
Testing Accuracy: 82.4%


### Hyperparameter tuning

In [19]:
logreg_paramgrid = [
    {'logreg__C': [100, 10, 1, 0.1, 0.01],
     'logreg__solver': ['lbfgs', 'sag', 'saga']}
]

In [20]:
logreg_pipe = Pipeline(steps=[('transform', transform_cols),
                              ('logreg', LogisticRegression(max_iter=3000))])
best_logreg = RandomizedSearchCV(logreg_pipe, logreg_paramgrid, n_iter=8, cv=5)

best_logreg.fit(X_train, y_train)

best_logreg.score(X_test, y_test)

c:\users\johng\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\johng\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\johng\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.8240103828682673

In [21]:
best_logreg.best_score_

0.8317432548156649

In [22]:
best_logreg.best_params_

{'logreg__solver': 'lbfgs', 'logreg__C': 10}

### Stratified k-Fold Cross Validation

Using values from hyperparameter tuning

In [24]:
strat_kfold(X, y, LogisticRegression(C=10, max_iter=4000))

Average training accuracy: 83.9% 
Average testing Accuracy: 82.97%

Highest testing accuracy: 83.31%


## Random Forest Pipeline

### Using `X_train` and `y_train`

In [25]:
rf_pipe = make_pipeline(transform_cols, RandomForestClassifier())

rf_pipe.fit(X_train, y_train)

training_score = round(rf_pipe.score(X_train, y_train)*100, 2)
testing_score = round(rf_pipe.score(X_test, y_test)*100, 2)

print(f'Training accuracy: {training_score}%',
      f'\nTesting Accuracy: {testing_score}%')

Training accuracy: 100.0% 
Testing Accuracy: 87.94%


### Hyperparameter Tuning

In [26]:
rf_paramgrid = [
    {'randfor__n_estimators': list(np.arange(100,501,20)),
     'randfor__max_depth': [10, 20, 30, None],
     'randfor__min_samples_split': [2, 4, 6],
     'randfor__min_samples_leaf': [1, 2, 4]}
]

In [27]:
rf_pipe = Pipeline(steps=[('transform', transform_cols),
                              ('randfor', RandomForestClassifier())])
best_rf = RandomizedSearchCV(rf_pipe, rf_paramgrid, cv=5, n_iter=30)

best_rf.fit(X_train, y_train)

best_rf.score(X_test, y_test)

0.8807268007787151

In [28]:
best_rf.best_score_

0.881944324964571

In [29]:
best_rf.best_params_

{'randfor__n_estimators': 480,
 'randfor__min_samples_split': 2,
 'randfor__min_samples_leaf': 1,
 'randfor__max_depth': None}

### Stratified k-Fold Cross Validation

Using the values acquired from hyperparameter tuning

In [31]:
strat_kfold(X, y, RandomForestClassifier())

Average training accuracy: 100.0% 
Average testing Accuracy: 88.21%

Highest testing accuracy: 88.76%


## Multi-Layer Perceptron

### Using `X_train` and `y_train`

In [32]:
nn_pipe = make_pipeline(transform_cols, MLPClassifier(max_iter=1000))

nn_pipe.fit(X_train, y_train)

training_score = round(nn_pipe.score(X_train, y_train)*100, 2)
testing_score = round(nn_pipe.score(X_test, y_test)*100, 2)

print(f'Training accuracy: {training_score}%',
      f'\nTesting Accuracy: {testing_score}%')

Training accuracy: 99.9% 
Testing Accuracy: 85.92%


### Hyperparameter Tuning

In [33]:
nn_paramgrid = [
    {'nn__hidden_layer_sizes': [(50, 50, 50), (25, 50, 25), (100,)],
     'nn__activation': ['logistic', 'relu'],
     'nn__learning_rate': ['constant', 'adaptive']}
]

In [34]:
nn_pipe = Pipeline(steps=[('transform', transform_cols),
                          ('nn', MLPClassifier(max_iter=2000))])
best_nn = RandomizedSearchCV(nn_pipe, nn_paramgrid, cv=5, n_iter=7)

best_nn.fit(X_train, y_train)

best_nn.score(X_test, y_test)

0.8597014925373134

In [35]:
best_nn.best_score_

0.8610136869006315

In [36]:
best_nn.best_params_

{'nn__learning_rate': 'constant',
 'nn__hidden_layer_sizes': (100,),
 'nn__activation': 'logistic'}

### Stratified k-Fold Cross Validation

Using the values from hyperparameter tuning

In [38]:
strat_kfold(X, y, MLPClassifier(max_iter=1000))

Average training accuracy: 99.87% 
Average testing Accuracy: 85.86%

Highest testing accuracy: 86.34%


# Remove Certain Columns to Possibly Improve Accuracy

In [39]:
# df['is_exchangeable'].value_counts(normalize=True)

In [40]:
# unequal_cols = ['engine_has_gas', 'has_warranty', 'state']

In [41]:
# maybe remove 'location_region', 'is_exchangeable'

In [42]:
# df_dropped = df.drop(unequal_cols, axis=1)
# df_dropped = df_dropped.drop(features, axis=1)

In [43]:
# X = df_dropped.drop('engine_type', axis=1)
# y = df_dropped['engine_type']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

# X_train.shape, y_train.shape

In [44]:
# # Columns with categorical data
# cat_cols = ['manufacturer_name', 'model_name', 'transmission', 'color', 'year_produced', 
#                'body_type', 'drivetrain', 'location_region']

# # Columns with numerical data
# num_cols = ['odometer_value', 'engine_capacity', 'price_usd', 'number_of_photos', 'up_counter', 'duration_listed']

# Regression

Predict how long the posting will be on the website.

## `OneHotEncoder()` on categorical columns, `StandardScaler( )` on numerical data

In [45]:
# Columns with categorical data
cat_cols = ['manufacturer_name', 'model_name', 'transmission', 'color', 'year_produced', 'engine_type',
               'body_type', 'has_warranty', 'state', 'drivetrain', 'location_region']

# Columns with numerical data
num_cols = ['odometer_value', 'engine_capacity', 'price_usd', 'number_of_photos', 'up_counter']

transform_cols = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols),
    (StandardScaler(), num_cols),
    remainder='passthrough'
)

## Set X and y and split into test/train

In [46]:
X = df.drop('duration_listed', axis=1)
y = df['duration_listed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, y_train.shape

((30816, 28), (30816,))

# Linear Regression

### Using `X_train` and `y_train`

In [47]:
linreg_pipe = make_pipeline(transform_cols, LinearRegression())

linreg_pipe.fit(X_train, y_train)

training_score = round(linreg_pipe.score(X_train, y_train)*100, 3)
testing_score = round(linreg_pipe.score(X_test, y_test)*100, 3)

print(f'Training accuracy: {training_score}%',
      f'\nTesting Accuracy: {testing_score}%')

Training accuracy: 54.118% 
Testing Accuracy: -1.1548511235979437e+23%


Testing accuracy is **very** bad

# Remove Certain Columns to Possibly Improve Accuracy

In [48]:
# df[num_cols].corr()

## `OneHotEncoder()` on categorical columns, `StandardScaler( )` on numerical data

In [49]:
# # Columns with categorical data
# cat_cols = ['manufacturer_name', 'model_name', 'transmission', 'color', 'year_produced', 'engine_type',
#                'body_type', 'drivetrain']

# # Columns with numerical data
# num_cols = ['odometer_value', 'engine_capacity', 'number_of_photos', 'up_counter', 'duration_listed']

# transform_cols = make_column_transformer(
#     (OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols),
#     (StandardScaler(), num_cols),
#     remainder='passthrough'
# )

In [50]:
# df['is_exchangeable'].value_counts(normalize=True)

In [51]:
# unequal_cols = ['engine_has_gas', 'has_warranty', 'state', 'location_region', 'is_exchangeable']
# df_dropped = df.drop(unequal_cols, axis=1)
# df_dropped = df_dropped.drop(features, axis=1)

In [52]:
# X = df_dropped.drop('price_usd', axis=1)
# y = df_dropped['price_usd']

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

# X_train.shape, y_train.shape

In [53]:
# linreg_pipe = make_pipeline(transform_cols, LinearRegression())

# linreg_pipe.fit(X_train, y_train)

# training_score = round(linreg_pipe.score(X_train, y_train)*100, 3)
# testing_score = round(linreg_pipe.score(X_test, y_test)*100, 3)

# print(f'Training accuracy: {training_score}%',
#       f'\nTesting Accuracy: {testing_score}%')

In [54]:
# strat_kfold(X, y, LinearRegression())

In [55]:
# # Columns with categorical data
# cat_cols = ['manufacturer_name', 'model_name', 'transmission', 'color', 'year_produced', 
#                'body_type', 'drivetrain', 'location_region']

# # Columns with numerical data
# num_cols = ['odometer_value', 'engine_capacity', 'price_usd', 'number_of_photos', 'up_counter', 'duration_listed']

## Temporarily Remove Categorical Data

In [56]:
# categorical_cols = ['manufacturer_name', 'model_name', 'transmission', 'color', 'year_produced', 'engine_fuel', 
#                     'engine_has_gas', 'engine_type', 'body_type', 'has_warranty', 'state', 'drivetrain', 
#                     'is_exchangeable', 'location_region', 'feature_0', 'feature_1', 'feature_2','feature_3', 
#                     'feature_4', 'feature_5', 'feature_6', 'feature_7','feature_8', 'feature_9']

# num_df = df.drop(categorical_cols, axis=1)

# num_df.head()

## Explore the Data

In [57]:
# def bar_plot(column):
#     x = df[column].value_counts().index.tolist()
#     y = df[column].value_counts().tolist()
#     sns.barplot(x=x,y=y)
#     plt.show()

In [58]:
# bar_plot('engine_fuel')
# bar_plot('engine_type')

In [59]:
# sns.boxplot(x=df['up_counter'])
# plt.show()

In [60]:
# sns.displot(df['price_usd'])
# plt.show()

In [61]:
# scaled_df.describe()

In [62]:
# df.describe()

In [63]:
# df.columns

In [64]:
# drop_cols = ['model_name','engine_fuel','engine_has_gas','has_warranty','state','is_exchangeable',
#              'feature_0','feature_1','feature_2','feature_3','feature_4','feature_5',
#              'feature_6','feature_7','feature_8','feature_9']

# dropped_df = df.drop(drop_cols, axis=1)

In [65]:
# dropped_df

In [66]:
# Check what kind of data is in each column
# dropped_df.dtypes

**Convert True/False Columns to 0/1**

In [67]:
# converted_df = df.copy()

In [68]:
# converted_df.head()

**Convert Categorical Data to Numeric Data**

In [69]:
# def categorical_to_num(df, column):
#     "Converts a list containing categorical values to a dictionary of the categorical name and an int64"
#     category = df[column].unique().tolist()          # Get the unique values from the column
#     idx = range(len(df[column].unique()))            # Create an index for each unique value from the column
#     category_dict = dict(zip(category, idx))         # Create dictionary of unique values and corresponding index
    
#     # https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict
#     df[column].replace(category_dict, inplace=True)  # Update the column in the DataFrame
    
#     return category_dict                             # Returns a dictionary

In [70]:
# for column in converted_df.columns:
#     if df.dtypes[column] == 'bool':
#         converted_df[column] = df[column].astype(np.int)    

In [71]:
# # Dictionary to keep all the information from each column as reference
# category_dicts = {}

# for column in converted_df.columns:
#     # Convert bool to int
#     if df.dtypes[column] == 'bool':
#         converted_df[column] = df[column].astype(np.int)
        
#     # Convert str to int
#     if converted_df.dtypes[column] == 'O': 
#         category_dicts[column] = categorical_to_num(converted_df, column) 

### Cleaned Data

In [72]:
# converted_df.head()

In [73]:
# Check to ensure that there is no categorical data
# converted_df.dtypes

In [74]:
# duplicate_val = dropped_df[dropped_df.duplicated()]
# print(f'There are {duplicate_val.shape[0]} duplicate rows')

In [75]:
# duplicate_val.head()

In [76]:
# num_df.corr()

In [77]:
# sns.set_theme(style="ticks")
# sns.pairplot(num_df)
# plt.show()

In [78]:
# num_df.head()

In [79]:
# stand_scale = preprocessing.StandardScaler()
# scaled_vals = stand_scale.fit_transform(num_df.values)
# scaled_df = pd.DataFrame(scaled_vals, columns=num_df.columns)

In [80]:
# scaled_df.corr()

In [81]:
# sns.set_theme(style="ticks")
# sns.pairplot(scaled_df)
# plt.show()